In [ ]:
#week_11 -  Which customers are closest?

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
branch = pd.read_csv("DSB Branches.csv")

In [3]:
branch.head(5)

,Branch,Branch Long,Branch Lat
0,London,-0.109863,51.481383
1,Oxford,-1.252441,51.727028
2,Nottingham,-1.142578,52.948638
3,Newcastle,-1.604004,54.965002


In [4]:
branch.shape

(4, 3)

In [5]:
branch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Branch       4 non-null      object 
 1   Branch Long  4 non-null      float64
 2   Branch Lat   4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 224.0+ bytes


In [6]:
cust = pd.read_csv("DSB Customer Locations.csv")

In [7]:
cust.head(5)

,Customer,Address Long,Address Lat
0,1,0.747070,51.549751
1,2,0.406494,51.957807
2,3,-0.142822,50.833698
3,4,-4.640350,50.335819
4,5,-1.593018,51.289406


In [8]:
cust.shape

(125, 3)

In [9]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Customer      125 non-null    int64  
 1   Address Long  125 non-null    float64
 2   Address Lat   125 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 3.1 KB


In [10]:
def haversine(lat1, lon1, lat2, lon2):
   
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = 3963 * c  # Earth radius in miles
    return round(distance, 2)

In [11]:
df = (cust.merge(branch,how='cross')
          .assign(Distance = lambda df_x :haversine(df_x['Address Lat'].astype(float),
                                                    df_x['Address Long'].astype(float),
                                                    df_x['Branch Lat'].astype(float),
                                                    df_x['Branch Long'].astype(float)))
          .sort_values('Distance',ascending = True)
          .drop_duplicates('Customer'))

In [12]:
df

,Customer,Address Long,Address Lat,Branch,Branch Long,Branch Lat,Distance
228,58,-0.043945,51.481383,London,-0.109863,51.481383,2.84
476,120,-0.156555,51.530960,London,-0.109863,51.481383,3.97
305,77,-1.164551,51.767840,Oxford,-1.252441,51.727028,4.70
472,119,-0.123596,51.404346,London,-0.109863,51.481383,5.36
480,121,-0.063171,51.559997,London,-0.109863,51.481383,5.80
...,...,...,...,...,...,...,...
100,26,1.494141,52.308479,London,-0.109863,51.481383,89.22
28,8,1.296387,52.616390,London,-0.109863,51.481383,98.69
177,45,-3.537598,50.792047,Oxford,-1.252441,51.727028,118.17
173,44,-4.130859,50.833698,Oxford,-1.252441,51.727028,139.01


In [13]:
df['Customer Priority'] = df.groupby('Branch')['Distance'].rank(ascending = True)

In [14]:
df

,Customer,Address Long,Address Lat,Branch,Branch Long,Branch Lat,Distance,Customer Priority
228,58,-0.043945,51.481383,London,-0.109863,51.481383,2.84,1.0
476,120,-0.156555,51.530960,London,-0.109863,51.481383,3.97,2.0
305,77,-1.164551,51.767840,Oxford,-1.252441,51.727028,4.70,1.0
472,119,-0.123596,51.404346,London,-0.109863,51.481383,5.36,3.0
480,121,-0.063171,51.559997,London,-0.109863,51.481383,5.80,4.0
...,...,...,...,...,...,...,...,...
100,26,1.494141,52.308479,London,-0.109863,51.481383,89.22,49.0
28,8,1.296387,52.616390,London,-0.109863,51.481383,98.69,50.0
177,45,-3.537598,50.792047,Oxford,-1.252441,51.727028,118.17,23.0
173,44,-4.130859,50.833698,Oxford,-1.252441,51.727028,139.01,24.0


In [15]:
df[df['Branch'] == 'London']

,Customer,Address Long,Address Lat,Branch,Branch Long,Branch Lat,Distance,Customer Priority
228,58,-0.043945,51.481383,London,-0.109863,51.481383,2.84,1.0
476,120,-0.156555,51.530960,London,-0.109863,51.481383,3.97,2.0
472,119,-0.123596,51.404346,London,-0.109863,51.481383,5.36,3.0
480,121,-0.063171,51.559997,London,-0.109863,51.481383,5.80,4.0
240,61,-0.263672,51.522416,London,-0.109863,51.481383,7.20,5.0
428,108,-0.291138,51.460852,London,-0.109863,51.481383,7.94,6.0
432,109,-0.140076,51.595842,London,-0.109863,51.481383,8.02,7.0
440,111,0.016479,51.390637,London,-0.109863,51.481383,8.31,8.0
272,69,-0.197754,51.371780,London,-0.109863,51.481383,8.48,9.0
216,55,-0.329590,51.508742,London,-0.109863,51.481383,9.65,10.0


In [16]:
cols = ['Branch','Branch Long','Branch Lat','Distance','Customer Priority','Customer','Address Long','Address Lat']

In [21]:
df.reindex(columns = cols)

,Branch,Branch Long,Branch Lat,Distance,Customer Priority,Customer,Address Long,Address Lat
228,London,-0.109863,51.481383,2.84,1.0,58,-0.043945,51.481383
476,London,-0.109863,51.481383,3.97,2.0,120,-0.156555,51.530960
305,Oxford,-1.252441,51.727028,4.70,1.0,77,-1.164551,51.767840
472,London,-0.109863,51.481383,5.36,3.0,119,-0.123596,51.404346
480,London,-0.109863,51.481383,5.80,4.0,121,-0.063171,51.559997
...,...,...,...,...,...,...,...,...
100,London,-0.109863,51.481383,89.22,49.0,26,1.494141,52.308479
28,London,-0.109863,51.481383,98.69,50.0,8,1.296387,52.616390
177,Oxford,-1.252441,51.727028,118.17,23.0,45,-3.537598,50.792047
173,Oxford,-1.252441,51.727028,139.01,24.0,44,-4.130859,50.833698


In [ ]:
df.to_csv("week_11_output.csv")